# Regression of Boston House Prices using Keras and TensorFlow

In this project, I will create a neural network model with Keras. I will also use scikit-learn to evaluate models using cross-validation. Finally, I will tune the network topology of models with Keras. My backend will be TensorFlow.

## Visualizing Dataset

In [1]:
import pandas as pd

# load dataset 
dataframe = pd.read_csv("housing.csv", delim_whitespace=True, header=None) 
dataset = dataframe.values

### Attributes

0. CRIM: per capita crime rate by town.
1. ZN: proportion of residential land zoned for lots over 25,000 sq.ft.
2. INDUS: proportion of non-retail business acres per town.
3. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
4. NOX: nitric oxides concentration (parts per 10 million).
5. RM: average number of rooms per dwelling.
6. AGE: proportion of owner-occupied units built prior to 1940.
7. DIS: weighted distances to ﬁve Boston employment centers.
8. RAD: index of accessibility to radial highways.
9. TAX: full-value property-tax rate per 10,000 dollars.
10. PTRATIO: pupil-teacher ratio by town. 
11. B: 1000(Bk−0.63)^2 where Bk is the proportion of blacks by town. 
12. LSTAT: Percent lower status of the population.
13. MEDV: Median value of owner-occupied homes in thousand dollars.

In [2]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


### As you can see, with lower crime rates, median house values increase. What else could we predict with Neural Networks using Keras and TensorFlow?

In [3]:
import seaborn as sns

new_df = pd.concat([dataframe[0], dataframe[13] * 1000], axis=1)
new_df.columns = ['Crime Rate', 'Median House Value']
sns.jointplot(x='Median House Value', y='Crime Rate', data=new_df)

## Develop a Baseline Neural Network Model

In [4]:
# Regression Example With Boston Dataset: Baseline
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# load dataset
dataframe = read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Using TensorFlow backend.


Baseline: 32.69 (23.27) MSE


## Lift Performance By Standardizing the Dataset

Knowing that each attribute in the Boston dataset measures in different scales, we could standardize everything to get better results. We are going to use scikit-learn's Pipeline feature within each fold of the cross-validation.

In [5]:
# Regression Example With Boston Dataset: Standardized
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# load dataset
dataframe = read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]
# define base model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Standardized: 29.33 (27.46) MSE


## Tuning the Neural Network Topology
Now that we have utilized k-folds for cross-validiation and standardization of datasets, let's work on expanding our Neural Network.

### Making our Neural Network even Deeper
Here we add more layers and neurons in each layer of our neural network.

In [6]:
# Regression Example With Boston Dataset: Standardized and Larger
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# load dataset
dataframe = read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]
# define the model
def larger_model():
	# create model
	model = Sequential()
	model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(6, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Larger: 23.22 (27.26) MSE


### Making our Neural Network even Wider
Now, we roughly double our neuron count from 13 to 20 inputs.

In [7]:
# Regression Example With Boston Dataset: Standardized and Wider
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# load dataset
dataframe = read_csv("housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:13]
Y = dataset[:,13]
# define wider model
def wider_model():
	# create model
	model = Sequential()
	model.add(Dense(20, input_dim=13, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Wider: 22.57 (29.61) MSE


## Conclusion

Our baseline model using Keras, TensorFlow, and SciKit-Learn scored: 32.60 (23.52) MSE. However, with standardizing our dataset, tuning our neural network by making our neural network deeper and wider, we were able to score a final 22.29 (29.54) MSE. That's over a $10,000 improvement in mean-squared-error!